# Interrupt Generator Demo

## Overview
This notebook demonstrates how to use the interrupt generator hardware block on the Xilinx KR260 FPGA. The interrupt generator provides two independent interrupt outputs that can be configured for either periodic (timer-based) or software-triggered operation.

## Functionality

This notebook provides a complete demonstration of the interrupt generator's capabilities:

### 1. **Hardware Setup**
   - Loads the FPGA bitstream overlay
   - Configures the clock frequency
   - Initializes interrupt instances for both interrupt outputs

### 2. **Interrupt Configuration**
   - **Interrupt1**: Configured for software-triggered interrupts (period1 = 0 disables periodic mode)
   - **Interrupt2**: Configured for periodic interrupts with a configurable period counter
   - Both interrupts are enabled via the Interrupt Enable Register (IER)

### 3. **Periodic Interrupts (Interrupt2)**
   - Demonstrates timer-based interrupts that fire automatically at regular intervals
   - Uses a period counter that counts clock cycles and generates an interrupt when it expires
   - Shows how to wait for and handle periodic interrupts asynchronously

### 4. **Software-Triggered Interrupts (Interrupt1)**
   - Demonstrates on-demand interrupt generation by writing to the trigger register
   - Shows how to manually trigger interrupts from software
   - Includes examples of single and multiple trigger scenarios

### 5. **Interrupt Handling**
   - Implements an async interrupt handler that waits for interrupts
   - Automatically clears interrupt status after handling
   - Demonstrates proper interrupt status register (ISR) management

## Register Map

The interrupt generator uses the following memory-mapped registers (byte addresses):

- **`period1 = 0`**: Period value for interrupt1 (32-bit). Set to 0 to disable periodic interrupts
- **`period2 = 4`**: Period value for interrupt2 (32-bit). Number of clock cycles between interrupts
- **`isr = 8`**: Interrupt Status Register (8-bit). Read to check status, write 1 to clear bits
  - Bit 0: Interrupt1 status
  - Bit 1: Interrupt2 status
- **`ier = 12`**: Interrupt Enable Register (8-bit). Controls which interrupts are enabled
  - Bit 0: Enable interrupt1 output
  - Bit 1: Enable interrupt2 output
- **`trigger = 16`**: Trigger Register (8-bit, write-only). Write 1 to trigger interrupts
  - Bit 0: Trigger interrupt1
  - Bit 1: Trigger interrupt2

## Usage Examples

### Software Trigger Usage:
1. Enable interrupt: `intr.write(ier, 3)` - enables both interrupts (0b11)
2. Trigger interrupt1: `intr.write(trigger, 1)` - writes 1 to bit 0
3. Wait for interrupt: `await interrupt_handler(intr_inst1, 1)`
4. Clear interrupt: `intr.write(isr, 1)` - writes 1 to bit 0 to clear

### Periodic Interrupt Usage:
1. Set period: `intr.write(period2, 100000000)` - sets period to 100M clock cycles
2. Enable interrupt: `intr.write(ier, 2)` - enables interrupt2 (bit 1)
3. Wait for interrupts: `await interrupt_handler(intr_inst2, 2)`

In [ ]:

from pynq import overlay, ps, PL, Interrupt
PL.reset()

In [ ]:
ps.Clocks.fclk0_mhz = 100
ov = overlay.Overlay("/lib/firmware/interrupt_demo.bit")
ov?

In [ ]:
intr = ov.interrupt_generator_0
intr?

In [ ]:
intr._interrupts

In [ ]:
intr_inst2 = Interrupt('interrupt_generator_0/interrupt2_out')
intr_inst1 = Interrupt('interrupt_generator_0/interrupt1_out') 

In [ ]:
period1 = 0
period2 = 4
isr = 8
ier = 12
trigger = 16


In [ ]:
async def interrupt_handler(interrupt_object, interrupt_bit):
    print("Handler task started. Waiting for interrupt...")

    # Wait for interrupt
    await interrupt_object.wait() 
    
    print("Interrupt received!")
    
    # Clear the interrupt (write 1 to the bit to clear it)
    intr.write(isr, interrupt_bit)


In [ ]:
intr.read(period2)

In [ ]:
# Configure interrupt1: Disable periodic interrupts (set period1 to 0)
# This allows us to use software triggering only
intr.write(period1, 0)
print("Period1 set to 0 (periodic interrupts disabled for interrupt1)")

# Configure interrupt2 for periodic interrupts
intr.write(period2, 100000000)

# Enable both interrupt1 (bit 0) and interrupt2 (bit 1) in IER
# Value 3 = 0b11 sets both bits
intr.write(ier, 3)  # Enable both interrupt1 and interrupt2
print("Interrupt1 and interrupt2 enabled")

In [ ]:
intr.read(isr)

In [ ]:
# Test interrupt2 (periodic)
count = 0

while count < 10:
    #loop.run_until_complete(handler_task)
    await interrupt_handler(intr_inst2, 2)  # Bit 1 (value 2) for interrupt2
    count += 1
    

print("Interrupt2 test finished.")

In [ ]:
# Software trigger for interrupt1
# Check current ISR status
print(f"ISR before trigger: {intr.read(isr)}")

# Trigger interrupt1 by writing 1 to bit 0 of trigger register
intr.write(trigger, 1)
print("Triggered interrupt1 via software")

# Wait a moment for the interrupt to be processed
import asyncio
await asyncio.sleep(0.1)

# Check ISR status after trigger
print(f"ISR after trigger: {intr.read(isr)}")

In [ ]:
# Test interrupt1 with software trigger
print("Testing interrupt1 with software trigger...")

# Clear any pending interrupts first
intr.write(isr, 1)  # Clear interrupt1 if set

# Trigger interrupt1
intr.write(trigger, 1)
print("Triggered interrupt1")

# Wait for the interrupt
await interrupt_handler(intr_inst1, 1)  # Bit 0 (value 1) for interrupt1

print("Interrupt1 test completed.")

In [ ]:
# Example: Trigger interrupt1 multiple times
print("Triggering interrupt1 multiple times...")

for i in range(5):
    # Clear previous interrupt
    intr.write(isr, 1)
    
    # Trigger interrupt1
    intr.write(trigger, 1)
    print(f"Trigger {i+1}: Triggered interrupt1")
    
    # Wait for interrupt
    await interrupt_handler(intr_inst1, 1)
    
    # Small delay between triggers
    await asyncio.sleep(0.1)

print("Multiple trigger test completed.")